In [1]:
# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time
from datetime import datetime
from datetime import timedelta
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", context="talk")

# ibl specific things
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses import behavior as behavioral_analyses

# set path for saving figures
figpath = 'figures' # put in the same folder, but add to .gitignore for now

Connecting anneurai@datajoint.internationalbrainlab.org:3306


In [2]:
# FIND THE DATA WE NEED
# BATCH 4

# get date for each weighing
weight_with_date = action.Weighing.proj('weight', session_date='DATE(weighing_time)')
weight_date = (dj.U('subject_uuid', 'session_date') & weight_with_date)
avg_weight_date = weight_date.aggr(weight_with_date, avg_weight='AVG(weight)')

batch4 = subject.Subject() & 'subject_nickname IN ("IBL_13", "IBL_14", "IBL_15", "IBL_16", "IBL_17")'
b = batch4 * (avg_weight_date & 'session_date BETWEEN "2018-08-05" AND "2018-09-12"')
df_batch4 = pd.DataFrame(b.fetch(as_dict=True, order_by='subject_nickname, session_date'))
df_batch4['session_day'] = (df_batch4['session_date'] - df_batch4['session_date'].min()).dt.days

# map onto conditions: see https://docs.google.com/spreadsheets/d/1jzaMEiuRPmbd6cu15IpWFyGqUkuHvlzF31b6ljAP5mc/edit#gid=420909701
bins = [np.datetime64('2018-08-05'), np.datetime64('2018-08-10'), np.datetime64('2018-08-17'), 
        np.datetime64('2018-08-28'), np.datetime64('2018-09-11')]
names = [1, 2, 4, 6]
df_batch4['condition'] = pd.cut(df_batch4.session_date, bins, labels=names)
tmpdf = df_batch4.loc[df_batch4.avg_weight > 30, :]

print(tmpdf)

# baseline correct with the first two days
for index, group in df_batch4.groupby(['subject_nickname']):
      df_batch4['avg_weight'][df_batch4.index.isin(group.index)] =  group['avg_weight'] / group[group['condition'] == 1]['avg_weight'].values.mean()

###### the other batch
batch2 = subject.Subject() & 'subject_nickname IN ("IBL_18", "IBL_19", "IBL_20", "IBL_21", "IBL_22", "IBL_23", "IBL_24", "IBL_25", "IBL_26", "IBL_27")'
b = batch2 * (avg_weight_date & 'session_date BETWEEN "2018-08-09" AND "2018-10-05"')
df_batch2 = pd.DataFrame(b.fetch(as_dict=True))
df_batch2['session_day'] = (df_batch2['session_date'] - df_batch2['session_date'].min()).dt.days

# map onto conditions
# Q: where is this info?
bins = [np.datetime64('2018-08-07'), np.datetime64('2018-08-24'), 
        np.datetime64('2018-09-17'), np.datetime64('2018-10-05')]
names = [1, 6, 5]
df_batch2['condition'] = pd.cut(df_batch2.session_date, bins, labels=names)
df_batch2 = df_batch2[df_batch2.session_day != 12] # hack

# baseline correct with the first two days
for index, group in df_batch2.groupby(['subject_nickname']):
     df_batch2['avg_weight'][df_batch2.index.isin(group.index)] = group['avg_weight']/ group[group['condition'] == 1]['avg_weight'].values.mean()
        
print(sorted(df_batch2.subject_nickname.unique()))
print(sorted(df_batch4.subject_nickname.unique()))

Empty DataFrame
Columns: [subject_uuid, session_date, subject_nickname, sex, subject_birth_date, ear_mark, subject_line, subject_source, protocol_number, subject_description, subject_ts, subject_strain, avg_weight, session_day, condition]
Index: []


/anaconda3/envs/iblenv/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['IBL_18', 'IBL_19', 'IBL_20', 'IBL_21', 'IBL_22', 'IBL_23', 'IBL_24', 'IBL_25', 'IBL_26', 'IBL_27']
['IBL_13', 'IBL_14', 'IBL_15', 'IBL_16', 'IBL_17']


/anaconda3/envs/iblenv/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
## PLOT #####
palette = sns.cubehelix_palette(4)
blue = sns.xkcd_palette(['azure'])
palette[0] = blue[0]

f, ax = plt.subplots(ncols=1, nrows=2, figsize=(5, 7), sharey=True)
sns.lineplot(x="session_day", y="avg_weight",  units='subject_nickname', estimator=None, 
             data=df_batch4, ax=ax[0], legend=False, color='lightgrey')
sns.lineplot(x="session_day", y="avg_weight", hue='condition', marker='o', ms=5,
             linewidth=0, units='subject_nickname', estimator=None, err_style='bars', 
             data=df_batch4, ax=ax[0], legend=False, palette=palette)
ax[0].set_xlabel("Days")    
ax[0].set_ylabel("Weight (normalized)")    
ax[0].set_ylim([0.8, 1.35])
ax[0].set_title('Cohort 4, n = 5 (15 months)', fontsize=16, fontstyle='italic')

ax[0].annotate('1 mL/day', xy=(0, 0.95), xytext=(0, 0.85), color=blue[0], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))
ax[0].annotate('2% CA HydroGel', xy=(5, 1.02), xytext=(5, 0.90), color=palette[1], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))
ax[0].annotate('4% CA HydroGel', xy=(13, 1.13), xytext=(13, 0.95), color=palette[2], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))
ax[0].annotate('6% CA HydroGel', xy=(23, 1.12), xytext=(23, 1), color=palette[3], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))


### second panel
palette = sns.cubehelix_palette(5)
sns.lineplot(x="session_day", y="avg_weight",  units='subject_nickname', estimator=None, 
             data=df_batch2, ax=ax[1], legend=False, color='lightgrey')
sns.lineplot(x="session_day", y="avg_weight", hue='condition', marker='o', ms=5,
             linewidth=0, units='subject_nickname', estimator=None, err_style='bars', 
             data=df_batch2, ax=ax[1], legend=False, palette=sns.cubehelix_palette(3))
ax[1].set_xlabel("Days")    
ax[1].set_ylabel("Weight (normalized)")  
ax[1].set_title('Cohort 5, n = 10 (15-16 months)', fontsize=16, fontstyle='italic')

ax[1].annotate('1% CA water', xy=(0, 1.1), xytext=(0, 1.2), color=palette[0], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))
ax[1].annotate('6% CA HydroGel', xy=(16, 1.03), xytext=(10, 1.13), color=palette[4], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))
ax[1].annotate('5% CA HydroGel', xy=(41, 1.1), xytext=(39, 1.18), color=palette[3], fontsize=14,
              arrowprops=dict(facecolor='lightgrey', shrink=0.05))

# SAVE
f.tight_layout()
sns.despine(trim=True)
f.savefig(os.path.join(figpath, "citricacid_study3.pdf"))
f.savefig(os.path.join(figpath, "citricacid_study3.png"), dpi=600)
plt.close('all')
print('done')

done
